<h1>Trabajdo Realizado para la Feria</h1>

<h2>Integrantes</h2>
<p>Luis Caballero 8-9231440<br>
Ángel Cilli<br>
Jaime Acosta</p>

<p><b>Idea o Concepto:</b>Trabajaremos un reconocedor de "personal" dentro del equipo, un Aruco distinto para Jaime Acosta y Luis Caballero, que cuando sean mostrados a la cámara el programa ejecutará distintos enfoques con Aruco.</p>

<h2>Detector del Personal</h2>

<p>Para el detector de personal usaremos el diccionario <b>DISC_4x4_1000</b>b><br>
Marcado ID 15 → Luis Caballero<br>
Marcador ID 13 → Jaime Acosta</p>

In [3]:
#IMPORTACIONES
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import pickle

#AruCO

OVERLAY_SIZE_PER = 1

# Check for camera calibration data
if not os.path.exists('calibration.pckl'):
    print("Necesitas Calibrar tu Cámara para un buen uso")
    exit()
else:
    f = open('calibration.pckl', 'rb')
    cameraMatrix, distCoeffs = pickle.load(f)
    f.close()
    if cameraMatrix is None or distCoeffs is None:
        print("Alguien salio mal. Recalibra tu cámara")
        exit()


# Create the ArUco dictionary and parameters:
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_1000) #Detector de Personal
aruco_dict2 = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_1000) #Imagen 3D
aruco_dict3 = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000) #Filtros
parameters = cv2.aruco.DetectorParameters()

# Create the video capture object:
video_capture = cv2.VideoCapture(0)
frame_index = 0;

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray_frame, aruco_dict, parameters=parameters)

    if ids is not None:
        for i in range(len(ids)):
            if ids[i] == 15:
                # Mensaje de bienvenida para Luis Caballero
                cv2.putText(img=frame, text="Bienvenido Luis", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                #aqui deberia ir lo de filtros
            elif ids[i] == 13:
                # Mensaje de bienvenida para Jaime Acosta
                cv2.putText(img=frame, text="Bienvenido Jaime", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                #aqui deberia ir lo del 3D
                
            else:
                #Mensaje de no esta registrado
                cv2.putText(img=frame, text="Quien tu eres!?", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                

    cv2.aruco.drawDetectedMarkers(frame, corners, ids)
    cv2.imshow('ArUco Marker and Face Detection', frame)
    

    if cv2.waitKey(20) & 0xFF == ord('c'):
        frame_name = "out/Parcial_frame_{}.png".format(frame_index)
        cv2.imwrite(frame_name, frame)
        frame_index += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import os
import pickle
import numpy as np
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
from objloader import OBJ  # Asumiendo que tienes un archivo objloader.py para cargar modelos obj

OVERLAY_SIZE_PER = 1

# Check for camera calibration data
if not os.path.exists('calibration.pckl'):
    print("You need to calibrate the camera before")
    exit()
else:
    f = open('calibration.pckl', 'rb')
    cameraMatrix, distCoeffs = pickle.load(f)
    f.close()
    if cameraMatrix is None or distCoeffs is None:
        print("Something went wrong. Recalibrate the camera")
        exit()


# Cargar el modelo 3D
obj = OBJ("path/to/your/model.obj")


def draw_points(img, pts):
    """ Draw the points in the image"""

    pts = np.int32(pts).reshape(-1, 2)

    # img = cv2.drawContours(img, [pts], -1, (255, 255, 0), -3)

    for p in pts:
        cv2.circle(img, (p[0], p[1]), 5, (255, 0, 255), -1)

    return img


def draw_augmented_overlay(pts_1, overlay_image, image):
    """Overlay the image 'overlay_image' onto the image 'image'"""

    # Define the squares of the overlay_image image to be drawn:
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])

    # Draw border to see the limits of the image:
    cv2.rectangle(overlay_image, (0, 0), (overlay_image.shape[1], overlay_image.shape[0]), (255, 255, 0), 10)

    # Create the transformation matrix:
    M = cv2.getPerspectiveTransform(pts_2, pts_1)

    # Transform the overlay_image image using the transformation matrix M:
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))
    #cv2.imshow("dst_image", dst_image)

    # Create the mask:
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

    # Compute bitwise conjunction using the calculated mask:
    image_masked = cv2.bitwise_and(image, image, mask=mask)
    #cv2.imshow("image_masked", image_masked)

    # Add the two images to create the resulting image:
    result = cv2.add(dst_image, image_masked)
    return result


# Create the dictionary object and the parameters:
#aruco_dictionary = cv2.aruco.Dictionary_get(cv2.aruco.DICT_7X7_250)
#parameters = cv2.aruco.DetectorParameters_create()

dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_7X7_250)
parameters =  cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

# Create video capture object 'capture' to be used to capture frames from the first connected camera:
capture = cv2.VideoCapture(0)

while True:
    # Capture frame by frame from the video capture object 'capture':
    ret, frame = capture.read()

    # We convert the frame to grayscale:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect markers:
    corners, ids, rejectedImgPoints = detector.detectMarkers(gray_frame) #cv2.aruco.detectMarkers(gray_frame, aruco_dictionary, parameters=parameters)

    # Draw detected markers:
    frame = cv2.aruco.drawDetectedMarkers(image=frame, corners=corners, ids=ids, borderColor=(0, 255, 0))

    # Draw rejected markers:
    # frame = cv2.aruco.drawDetectedMarkers(image=frame, corners=rejectedImgPoints, borderColor=(0, 0, 255))

    if ids is not None:
        # rvecs and tvecs are the rotation and translation vectors respectively, for each of the markers in corners.
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 1, cameraMatrix, distCoeffs)

        for rvec, tvec in zip(rvecs, tvecs):
            # Note: The marker coordinate system is centered on the center of the marker
            # The coordinates of the four corners of the marker (in its own coordinate system) are:
            # 1: (-markerLength/2, markerLength/2, 0)
            # 2: (markerLength/2, markerLength/2, 0)
            # 3: (markerLength/2, -markerLength/2, 0)
            # 4: (-markerLength/2, -markerLength/2, 0)
            # Define the points where you want the image to be overlaid (remember: marker coordinate system):
            desired_points = np.float32(
                [[-1 / 2, 1 / 2, 0], [1 / 2, 1 / 2, 0], [1 / 2, -1 / 2, 0], [-1 / 2, -1 / 2, 0]]) * OVERLAY_SIZE_PER

            # Proyectar puntos y overlay del modelo 3D
            projected_desired_points, jac = cv2.projectPoints(desired_points, rvecs, tvecs, cameraMatrix, distCoeffs)

            # Obtener la matriz de vista utilizando los parámetros de cámara
            view_matrix = cv2.Rodrigues(rvec)[0]

            # Renderizar el modelo 3D
            glLoadIdentity()
            glLoadMatrixd(view_matrix)
            glTranslatef(tvec[0][0], tvec[0][1], tvec[0][2])  # Ajusta según las coordenadas del marcador

            glEnable(GL_DEPTH_TEST)
            obj.render()

            glDisable(GL_DEPTH_TEST)

  # Display the resulting augmented frame:
    cv2.imshow('frame', frame)

    # Press 'q' to exit:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything:
capture.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'OpenGL'